# Modelagem - Previsão de Doenças Cardíacas


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import warnings
warnings.filterwarnings('ignore')


In [4]:
print("="*60)
print("ANÁLISE DE PREVISÃO DE DOENÇAS CARDÍACAS")
print("="*60)

df = pd.read_csv('../data/heart_disease.csv')
print(f"\nDimensões do dataset: {df.shape}")
print(f"\nPrimeiras linhas:")
print(df.head())


ANÁLISE DE PREVISÃO DE DOENÇAS CARDÍACAS

Dimensões do dataset: (200, 14)

Primeiras linhas:
   age  sex  chest_pain_type  resting_bp  cholesterol  fasting_bs  \
0   63    1                3         145          233           1   
1   37    1                2         130          250           0   
2   41    0                1         130          204           0   
3   56    1                1         120          236           0   
4   57    0                0         120          354           0   

   resting_ecg  max_hr  exercise_angina  oldpeak  st_slope  num_vessels  \
0            0     150                0      2.3         0            0   
1            1     187                0      3.5         0            0   
2            0     172                0      1.4         2            0   
3            1     178                0      0.8         2            0   
4            1     163                1      0.6         2            0   

   thalassemia  target  
0            1  

In [5]:
print(f"\n{'─'*60}")
print("ANÁLISE EXPLORATÓRIA")
print("─"*60)

print(f"\nValores ausentes: {df.isnull().sum().sum()}")
print(f"\nDistribuição da variável target:")
print(df['target'].value_counts())



────────────────────────────────────────────────────────────
ANÁLISE EXPLORATÓRIA
────────────────────────────────────────────────────────────

Valores ausentes: 0

Distribuição da variável target:
target
0    139
1     61
Name: count, dtype: int64


In [6]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")



Tamanho do conjunto de treino: (160, 13)
Tamanho do conjunto de teste: (40, 13)


In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [9]:
print(f"\n{'─'*60}")
print("TREINAMENTO DOS MODELOS")
print("─"*60)

models = {
    'Regressão Logística': LogisticRegression(max_iter=1000, random_state=42),
    'Árvore de Decisão': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

trained_models = {}

for name, model in models.items():
    print(f"\nTreinando {name}...")
    model.fit(X_train_scaled, y_train)
    trained_models[name] = model
    print(f"{name} treinado!")



────────────────────────────────────────────────────────────
TREINAMENTO DOS MODELOS
────────────────────────────────────────────────────────────

Treinando Regressão Logística...
Regressão Logística treinado!

Treinando Árvore de Decisão...
Árvore de Decisão treinado!

Treinando Random Forest...
Random Forest treinado!


In [10]:
print(f"\n{'─'*60}")
print("AVALIAÇÃO DOS MODELOS")
print("─"*60)

results = []

for name, model in trained_models.items():
    y_pred = model.predict(X_test_scaled)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    results.append({
        'Modelo': name,
        'Acurácia': accuracy,
        'Precisão': precision,
        'Recall': recall,
        'F1-Score': f1
    })

results_df = pd.DataFrame(results)
results_df = results_df.round(4)
print(f"\n{results_df.to_string(index=False)}")



────────────────────────────────────────────────────────────
AVALIAÇÃO DOS MODELOS
────────────────────────────────────────────────────────────

             Modelo  Acurácia  Precisão  Recall  F1-Score
Regressão Logística     0.850    0.8000  0.6667    0.7273
  Árvore de Decisão     0.775    0.6364  0.5833    0.6087
      Random Forest     0.800    0.7000  0.5833    0.6364


In [11]:
best_model_name = results_df.loc[results_df['F1-Score'].idxmax(), 'Modelo']
best_model = trained_models[best_model_name]

print(f"\n{'─'*60}")
print(f"MELHOR MODELO: {best_model_name}")
print("─"*60)

joblib.dump(best_model, '../modelo_final.pkl')
joblib.dump(scaler, '../scaler.pkl')

print(f"\nModelo e scaler salvos com sucesso!")



────────────────────────────────────────────────────────────
MELHOR MODELO: Regressão Logística
────────────────────────────────────────────────────────────

Modelo e scaler salvos com sucesso!


In [13]:
print(f"\n{'='*60}")
print("TESTE COM NOVO PACIENTE")
print("="*60)

loaded_model = joblib.load('../modelo_final.pkl')
loaded_scaler = joblib.load('../scaler.pkl')

novo_paciente = {
    'age': 55,
    'sex': 1,
    'chest_pain_type': 2,
    'resting_bp': 140,
    'cholesterol': 260,
    'fasting_bs': 0,
    'resting_ecg': 0,
    'max_hr': 150,
    'exercise_angina': 1,
    'oldpeak': 2.5,
    'st_slope': 1,
    'num_vessels': 1,
    'thalassemia': 3
}

novo_paciente_df = pd.DataFrame([novo_paciente])
novo_paciente_scaled = loaded_scaler.transform(novo_paciente_df)
previsao = loaded_model.predict(novo_paciente_scaled)
probabilidade = loaded_model.predict_proba(novo_paciente_scaled)

print(f"\nCaracterísticas do paciente:")
for key, value in novo_paciente.items():
    print(f"  {key}: {value}")

print(f"\n{'─'*60}")
print(f"\nResultado: {'POSITIVO para doença cardíaca' if previsao[0] == 1 else 'NEGATIVO para doença cardíaca'}")
print(f"\nProbabilidade de NÃO ter doença: {probabilidade[0][0]:.2%}")
print(f"Probabilidade de TER doença: {probabilidade[0][1]:.2%}")
print(f"\n{'='*60}")



TESTE COM NOVO PACIENTE

Características do paciente:
  age: 55
  sex: 1
  chest_pain_type: 2
  resting_bp: 140
  cholesterol: 260
  fasting_bs: 0
  resting_ecg: 0
  max_hr: 150
  exercise_angina: 1
  oldpeak: 2.5
  st_slope: 1
  num_vessels: 1
  thalassemia: 3

────────────────────────────────────────────────────────────

Resultado: NEGATIVO para doença cardíaca

Probabilidade de NÃO ter doença: 92.63%
Probabilidade de TER doença: 7.37%

